In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


from keys import Trip_advisor_key
from keys import geoapify_key

In [3]:

# Definizione dei distretti e delle coordinate
Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236, 43.087586], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009, -79.081206]  
})


# Definizione del dataframe dei ristoranti
Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Restaurant Address', 'Lat', 'Lng'])

# Preparazione dello script per il primo distretto scelto
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            print('Errore nella gestione dei dati del ristorante.')


In [5]:
Restaurants_df.head()

Location        Restaurant Name  \
0  Toronto Distillery District       Archeo Trattoria   
1  Toronto Distillery District              El Catrin   
2  Toronto Distillery District   Oyster House & grill   
3  Toronto Distillery District           Pure Spirits   
4  Toronto Distillery District  The Fermenting Cellar   

                                  Restaurant Address        Lat        Lng  
0  31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...  43.650682 -79.359630  
1       Mill Street, Old Toronto, ON M5A 3C4, Canada  43.650693 -79.359245  
2   Tank House Lane, Old Toronto, ON M5A 3C4, Canada  43.650350 -79.358811  
3  17 Tank House Lane, Old Toronto, ON M5A 3C4, C...  43.650395 -79.358754  
4  28 Distillery Lane, Old Toronto, ON M5A 3C4, C...  43.649806 -79.360538

In [17]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [11]:


url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery=El%20Catrin&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{
  "data": [
    {
      "location_id": "13483140",
      "name": "Catrin",
      "address_obj": {
        "street1": "Calle 47 # 463B",
        "street2": "Centro",
        "city": "Merida",
        "state": "Yucatan",
        "country": "Mexico",
        "postalcode": "97000",
        "address_string": "Calle 47 # 463B Centro, Merida 97000 Mexico"
      }
    },
    {
      "location_id": "4556927",
      "name": "El Catrin",
      "address_obj": {
        "street1": "18 Tank House Lane",
        "street2": "Distillery District",
        "city": "Toronto",
        "state": "Ontario",
        "country": "Canada",
        "postalcode": "M5A 3C4",
        "address_string": "18 Tank House Lane Distillery District, Toronto, Ontario M5A 3C4 Canada"
      }
    },
    {
      "location_id": "10276404",
      "name": "El Catrin",
      "address_obj": {
        "street1": "rue de Richemont 7-9",
        "city": "Geneva",
        "country": "Switzerland",
        "postalcode": "1202",
       